In [1]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

print(f"CUDA available: {cuda_available}")

CUDA available: True


In [1]:
import os
import pandas as pd
import h3
import folium
from collections import defaultdict
import tqdm
from tqdm.auto import tqdm
import random
from itertools import combinations
import random
from sentence_transformers import SentenceTransformer, losses, SentencesDataset, InputExample, models
from transformers import AutoTokenizer, RobertaModel, RobertaConfig
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import torch
import shutil
import zipfile
import numpy as np
from annoy import AnnoyIndex
import psutil

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
memory_info = psutil.virtual_memory()

print(f"Total Memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used Memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Percentage Used: {memory_info.percent}%")

Total Memory: 29.39 GB
Available Memory: 23.17 GB
Used Memory: 5.76 GB
Percentage Used: 21.2%


In [25]:
df = pd.read_csv('my_data.csv')

print(df.head())

                                             Address   Latitude  Longitude
0          0 14 4 th cross road phase 2 amrita nagar  12.893634  77.678591
1                  0 29 janatha colony doddakannalli  12.913099  77.698461
2                           0 81 sri sai fancy store  12.883179  77.695634
3   0 a wipro it sez project kodathi village vart...  12.898700  77.713861
4   0 birlamant shriam chirping wood villiyammant...  12.908984  77.668186


In [26]:
from transformers import AutoTokenizer, AutoModel
import torch
import os

# Define the directory where your model files are located
model_dir = "/home/devanapalli.ravi/disk_1/paper/data/untrained_models/distilroberta-base/"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the model
model = AutoModel.from_pretrained(model_dir)

# Optionally, verify that the model is loaded correctly
print("Model and tokenizer loaded successfully")




Model and tokenizer loaded successfully


In [5]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout)

In [28]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# # Example usage
# example_text = "This is an example sentence."
# embedding = get_embedding(example_text)
# print(f"Embedding for the example text: {embedding}")

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196972 entries, 0 to 196971
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Address    196972 non-null  object 
 1   Latitude   196972 non-null  float64
 2   Longitude  196972 non-null  float64
dtypes: float64(2), object(1)
memory usage: 4.5+ MB


In [40]:
addresses = df['Address'].tolist()
lat_lngs = df[['Latitude', 'Longitude']].values.tolist()

print("Addresses:", addresses[:10])
print("Lat/Lng:", lat_lngs[:10])

Addresses: [' 0 14 4 th cross road phase 2 amrita nagar', ' 0 29 janatha colony doddakannalli', ' 0 81 sri sai fancy store', ' 0 a wipro it sez project kodathi village varthur hobli', ' 0 birlamant shriam chirping wood villiyammant layout haralur enclave layout bengal ura 560035 ', ' 0 fast n fresh super market junnasandra road and sajapur main road', ' 0 janatha colony carmelaram', ' 0 kodathi get', ' 0 kpr elite kaikondrahalli', ' 0 near aet circle lakasmi hardvear doddakannalli sarjapur main road bengaluru 560035 ']
Lat/Lng: [[12.8936336, 77.6785909], [12.913098800000002, 77.698461], [12.883179, 77.6956343], [12.8987, 77.7138608], [12.9089839, 77.6681863], [12.9082538, 77.6834946], [12.9061864, 77.7034767], [12.8922406, 77.7187151], [12.9085819, 77.676254], [12.9091267, 77.6934067]]


In [30]:
# addresses = addresses[:10000]
# lat_lngs = lat_lngs[:10000]

In [10]:
len(addresses)

10000

In [41]:
def generate_h3_cells(lat_lngs, resolution):
    return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

resolution = 10
h3_cells = generate_h3_cells(lat_lngs, resolution)
len(h3_cells)

196972

In [20]:
# def is_valid_h3_cell(cell):
#     try:
#         h3.h3_get_resolution(cell)
#         return True
#     except h3.H3CellError:
#         return False

# def generate_h3_cells(lat_lngs, resolution):
#     return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

# def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
#     cell_to_points = defaultdict(list)
#     h3_cells_set = set(h3_cells)

#     for index, cell in enumerate(h3_cells):
#         if is_valid_h3_cell(cell):
#             cell_to_points[cell].append(index)

#     triplets = []
#     for cell, points in tqdm(cell_to_points.items(), desc="Generating Triplets"):
#         if len(points) > 1:
#             current_resolution = h3.h3_get_resolution(cell)
#             if current_resolution >= resolution:
#                 try:
#                     parent_cell = h3.h3_to_parent(cell, resolution - 1)
#                 except (h3.H3CellError, h3.H3ResolutionError):
#                     continue

#                 rings = []
#                 for ring_level in range(1, D + 1):
#                     if ring_level == 1:
#                         ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1)
#                     else:
#                         previous_ring = h3.k_ring(parent_cell, ring_level)
#                         ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
#                     rings.append(ring)

#             for point in points:
#                 positive_points = [p for p in points if p != point] ## list for possible positive pair for a given anchor. A -- p1 , p2 , ... , pN
#                 if positive_points:
#                     sampled_positives = random.sample(positive_points, min(K, len(positive_points))) ## for a given anchor , choosing random "K" positive points to form paird. A -- p1 , p2 , ... , pK
#                     for positive_sample in sampled_positives: # for a given A -- P1 --- forming conectric rings upto depth "D"
#                         for ring_level in range(1, D + 1):
#                             valid_negative_samples = set()
#                             for neighbor in rings[ring_level - 1]:
#                                 children = h3.h3_to_children(neighbor, resolution)
#                                 for child in children:
#                                     if child in h3_cells_set:
#                                         valid_negative_samples.add(child)
#                                         if len(valid_negative_samples) >= 10000: ## A--p1 --- n1,n2,...,n50 (for each ring)
#                                             break
#                                 if len(valid_negative_samples) >= 10000:
#                                     break
#                             if valid_negative_samples:
#                                 sampled_negatives = random.sample(list(valid_negative_samples), min(T, len(valid_negative_samples)))   ## A--p1--n1,n2,n3,...,nT
#                                 for neg in sampled_negatives:
#                                     neg_index = h3_cells.index(neg)
#                                     triplets.append((point, positive_sample, neg_index, ring_level))

#     return triplets


In [ ]:
print("hi")

In [19]:
# def is_valid_h3_cell(cell):
#     try:
#         h3.h3_get_resolution(cell)
#         return True
#     except h3.H3CellError:
#         return False

# def generate_h3_cells(lat_lngs, resolution):
#     return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

# def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
#     cell_to_points = defaultdict(list)
#     h3_cells_set = set(h3_cells)

#     for index, cell in enumerate(h3_cells):
#         if is_valid_h3_cell(cell):
#             cell_to_points[cell].append(index)

#     triplets = []
#     for cell, points in tqdm(cell_to_points.items(), desc="Generating Triplets"):
#         if len(points) > 1:
#             current_resolution = h3.h3_get_resolution(cell)
#             if current_resolution >= resolution:
#                 try:
#                     parent_cell = h3.h3_to_parent(cell, resolution - 1)
#                 except (h3.H3CellError, h3.H3ResolutionError):
#                     continue

#                 rings = []
#                 for ring_level in range(1, D + 1):
#                     if ring_level == 1:
#                         ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1)
#                     else:
#                         previous_ring = h3.k_ring(parent_cell, ring_level)
#                         ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
#                     rings.append(ring)

#                 # Precompute valid negative samples for each ring level
#                 negative_samples_per_ring = {level: set() for level in range(1, D + 1)}
#                 for ring_level, ring in enumerate(rings, start=1):
#                     for neighbor in ring:
#                         children = h3.h3_to_children(neighbor, resolution)
#                         for child in children:
#                             if child in h3_cells_set:
#                                 negative_samples_per_ring[ring_level].add(child)
#                             if len(negative_samples_per_ring[ring_level]) >= 100000:
#                                 break
#                         if len(negative_samples_per_ring[ring_level]) >= 100000:
#                             break

#                 # Generate triplets
#                 for point in points:
#                     positive_points = [p for p in points if p != point]
#                     if positive_points:
#                         sampled_positives = random.sample(positive_points, min(K, len(positive_points)))
#                         for positive_sample in sampled_positives:
#                             for ring_level in range(1, D + 1):
#                                 valid_negative_samples = list(negative_samples_per_ring[ring_level])
#                                 if valid_negative_samples:
#                                     sampled_negatives = random.sample(valid_negative_samples, min(T, len(valid_negative_samples)))
#                                     for neg in sampled_negatives:
#                                         neg_index = h3_cells.index(neg)
#                                         triplets.append((point, positive_sample, neg_index, ring_level))

#     return triplets


In [43]:
import h3
import random
from tqdm import tqdm
from collections import defaultdict

def is_valid_h3_cell(cell):
    try:
        h3.h3_get_resolution(cell)
        return True
    except h3.H3CellError:
        return False

def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
    cell_to_points = defaultdict(list)
    h3_cells_set = set(h3_cells)

    for index, cell in enumerate(h3_cells):
        if is_valid_h3_cell(cell):
            cell_to_points[cell].append(index)

    triplets = []
    for cell, points in tqdm(cell_to_points.items(), desc="Generating Triplets"):
        if len(points) > 1:
            current_resolution = h3.h3_get_resolution(cell)
            if current_resolution >= resolution:
                try:
                    parent_cell = h3.h3_to_parent(cell, resolution - 1)
                except (h3.H3CellError, h3.H3ResolutionError):
                    continue

                rings = []
                for ring_level in range(1, D + 1):
                    if ring_level == 1:
                        ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1)
                    else:
                        previous_ring = h3.k_ring(parent_cell, ring_level)
                        ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
                    rings.append(ring)

            valid_negative_samples_by_level = {ring_level: set() for ring_level in range(1, D + 1)}
            for ring_level, ring in enumerate(rings, start=1):
                for neighbor in ring:
                    children = h3.h3_to_children(neighbor, resolution)  # Convert set to list
                    random.shuffle(list(children))  
                    for child in children:
                        if child in h3_cells_set:
                            valid_negative_samples_by_level[ring_level].add(child)
                            if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                                break
                    if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                        break

            for point in points:
                positive_points = [p for p in points if p != point]
                if positive_points:
                    sampled_positives = random.sample(positive_points, min(K, len(positive_points)))
                    for positive_sample in sampled_positives:
                        for ring_level in range(1, D + 1):
                            valid_negative_samples = valid_negative_samples_by_level[ring_level]
                            if valid_negative_samples:
                                sampled_negatives = random.sample(list(valid_negative_samples), min(T, len(valid_negative_samples)))
                                for neg in sampled_negatives:
                                    neg_index = h3_cells.index(neg)
                                    triplets.append((point, positive_sample, neg_index, ring_level))

    return triplets



In [44]:
K = 2 ##upper bound is 50
T = 1
D = 3
triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)
print("Generated triplets:", len(triplets))

Generating Triplets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4524/4524 [10:45<00:00,  7.01it/s]

Generated triplets: 1165144


In [16]:
K = 2 ##upper bound is 50
T = 1
D = 3
triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)
print("Generated triplets:", len(triplets))

Generating Triplets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4524/4524 [11:19<00:00,  6.66it/s]

Generated triplets: 1165144


In [18]:
K = 2 ##upper bound is 10000
T = 1
D = 3
triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)
print("Generated triplets:", len(triplets))

Generating Triplets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4524/4524 [10:46<00:00,  7.00it/s]

Generated triplets: 1165144


In [21]:
K = 2 ##upper bound is 10000 (un)
T = 1
D = 3
triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)
print("Generated triplets:", len(triplets))

Generating Triplets: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4524/4524 [13:40<00:00,  5.52it/s]

Generated triplets: 1165144


In [19]:
def add_hexagons_to_map(h3_cells, color, m):
    for cell in h3_cells:
        boundary = h3.h3_to_geo_boundary(cell)
        boundary = [(lat, lng) for lat, lng in boundary]
        folium.Polygon(
            locations=boundary,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.0003,
            weight=2
        ).add_to(m)

def visualize_triplets_with_grid(lat_lngs, triplets, h3_cells, resolution, D):

    average_lat = sum(lat for lat, lng in lat_lngs) / len(lat_lngs)
    average_lng = sum(lng for lat, lng in lat_lngs) / len(lat_lngs)
    m = folium.Map(location=[average_lat, average_lng], zoom_start=13)

    colors = ['orange', 'red', 'purple', 'blue']

    for triplet in triplets:

        anchor_idx, positive_idx, negative_idx, ring_level = triplet


        anchor_coord = lat_lngs[anchor_idx]
        positive_coord = lat_lngs[positive_idx]
        negative_coord = lat_lngs[negative_idx]

        folium.Marker(location=anchor_coord, popup='Anchor', icon=folium.Icon(color='blue')).add_to(m)
        folium.Marker(location=positive_coord, popup='Positive', icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=negative_coord, popup='Negative', icon=folium.Icon(color='red')).add_to(m)

        folium.PolyLine(locations=[anchor_coord, positive_coord], color='red' , weight=1.5).add_to(m)
        folium.PolyLine(locations=[anchor_coord, negative_coord], color='blue', weight=1.5).add_to(m)

        anchor_cell = h3_cells[anchor_idx]
        add_hexagons_to_map([anchor_cell], 'blue', m)

        positive_cell = h3_cells[positive_idx]
        add_hexagons_to_map([positive_cell], 'green', m)

        parent_cell = h3.h3_to_parent(anchor_cell, resolution - 1)
        add_hexagons_to_map([parent_cell], 'yellow', m)

        rings = []
        for ring_level in range(1, D + 1):
            if ring_level == 1:
                ring = h3.k_ring(parent_cell, 1) - {parent_cell}
            else:
                previous_ring = h3.k_ring(parent_cell, ring_level - 1)
                ring = h3.k_ring(parent_cell, ring_level) - previous_ring
            rings.append(ring)

        for ring_level in range(1, D + 1):
            ring = rings[ring_level - 1]
            add_hexagons_to_map(ring, colors[ring_level - 1], m)

    m.save('.html')
    return m

In [ ]:

if triplets:
    visualize_triplets_with_grid(lat_lngs, triplets[90:200], h3_cells, resolution, D+1)

In [12]:
print("triplets:", triplets[:10])

triplets: [(0, 775, 128, 1), (0, 775, 150, 2), (0, 775, 22, 3), (775, 0, 53, 1), (775, 0, 149, 2), (775, 0, 710, 3), (1, 43, 308, 1), (1, 43, 566, 2), (1, 43, 160, 3), (1, 632, 326, 1)]


In [ ]:
# # Initialize the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
# config = RobertaConfig()
# roberta_model = RobertaModel(config)

In [13]:
save_directory = './temp_roberta_model'
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./temp_roberta_model/tokenizer_config.json',
 './temp_roberta_model/special_tokens_map.json',
 './temp_roberta_model/vocab.json',
 './temp_roberta_model/merges.txt',
 './temp_roberta_model/added_tokens.json',
 './temp_roberta_model/tokenizer.json')

In [14]:
# Create a SentenceTransformer-compatible transformer model
word_embedding_model = models.Transformer(model_name_or_path=save_directory, max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())



In [15]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [16]:
# Prepare training examples
train_examples = [
    InputExample(texts=[addresses[anchor], addresses[positive], addresses[negative]], label=ring_level)
    for anchor, positive, negative, ring_level in triplets
]

train_dataset = SentencesDataset(train_examples, model)

In [17]:

# Custom collate function
def custom_collate_fn(batch):
    texts = []
    ring_levels = []
    for example in batch:
        texts.append(example.texts)
        ring_levels.append(example.label)
    return {'texts': texts, 'ring_levels': torch.tensor(ring_levels)}

# DataLoader with custom collate function
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32, collate_fn=custom_collate_fn)

In [ ]:
for batch in train_dataloader:
    print(batch)
    break

In [18]:
class ModifiedCustomTripletLoss(losses.TripletLoss):
    def __init__(self, model, margin=1.0):
        super(ModifiedCustomTripletLoss, self).__init__(model)
        self.margin = margin

    def forward(self, sentence_features, ring_levels):
        embeddings = [self.model(sentence_feature)['sentence_embedding'] for sentence_feature in sentence_features]
        anchor, positive, negative = embeddings
        margin_adjusted = self.margin * ring_levels
        return F.triplet_margin_loss(anchor, positive, negative, margin=margin_adjusted.mean())

In [ ]:
margin = 2.0
train_loss = ModifiedCustomTripletLoss(model=model, margin=margin)

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=10)

print("Training complete.")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
margin = 2.0
train_loss = ModifiedCustomTripletLoss(model=model, margin=margin)

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=10)

print("Training complete.")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Training complete.


In [21]:
import os
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

save_directory = './trained_roberta_model'
os.makedirs(save_directory, exist_ok=True)

def save_model_weights(model, save_directory):
    model_path = os.path.join(save_directory, 'pytorch_model.bin')
    torch.save(model.state_dict(), model_path)
    print("Model weights saved successfully.")

print("Saving the model weights...")
with tqdm(total=1, desc="Saving Model Weights") as pbar:
    save_model_weights(model, save_directory)
    pbar.update(1)

print("Model weights saved successfully.")

Saving the model weights...


Saving Model Weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.90s/it]

Model weights saved successfully.
Model weights saved successfully.


In [22]:
save_directory = './trained_roberta_model'
os.makedirs(save_directory, exist_ok=True)

# Save only the model weights
def save_model_weights(model, save_directory):
    model_path = os.path.join(save_directory, 'pytorch_model.bin')
    torch.save(model.state_dict(), model_path)
    print("Model weights saved successfully.")

print("Saving the model weights...")
with tqdm(total=1, desc="Saving Model Weights") as pbar:
    save_model_weights(model, save_directory)
    pbar.update(1)

print("Model weights saved successfully.")


Saving the model weights...


Saving Model Weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.16s/it]

Model weights saved successfully.
Model weights saved successfully.


In [30]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [2]:
print('hi')

hi


In [31]:
# import os
# import torch
# from sentence_transformers import SentenceTransformer, models

# # Specify the directory where the weights are saved
# load_directory = './trained_roberta_model'
# model_weights_path = os.path.join(load_directory, 'pytorch_model.bin')

# # Initialize the transformer model and pooling layer
# word_embedding_model = models.Transformer(model_name_or_path=save_directory, max_seq_length=256)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# # Create the SentenceTransformer model
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# # Load the model weights
# model.load_state_dict(torch.load(model_weights_path))

# print("Model weights loaded successfully.")


Model weights loaded successfully.


In [ ]:
# from sentence_transformers import SentenceTransformer

# load_directory = './trained_roberta_model'
# loaded_model = SentenceTransformer(load_directory)

# print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
# import shutil
# import zipfile
# shutil.make_archive('trained_roberta_model', 'zip', save_directory)

# from google.colab import files
# files.download('trained_roberta_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from sentence_transformers import SentenceTransformer

# # Define the directory where the model is saved
# load_directory = './trained_roberta_model'

# # Load the SentenceTransformer model
# model = SentenceTransformer(load_directory)

# print("Model loaded successfully.")


Model loaded successfully.


# ANN


In [15]:
from google.colab import drive

ModuleNotFoundError: No module named 'google'

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip the Model File
zip_path = '/content/drive/MyDrive/Models/Roberta-Model/trained_roberta_model.zip'
extract_dir = '/content/trained_roberta_model'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Load the Model
model = SentenceTransformer(extract_dir)
print("Model loaded successfully.")

Model loaded successfully.


In [35]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [23]:
import numpy as np
from annoy import AnnoyIndex
from scipy.stats import gaussian_kde
from math import radians, sin, cos, sqrt, atan2
from sentence_transformers import SentenceTransformer


In [24]:
# Tokenizer
tokenizer = model.tokenizer

In [25]:
from tqdm.auto import tqdm

In [26]:
def get_embedding(text):
    return model.encode(text)

# Generate embeddings for the dataset
embeddings = np.vstack([get_embedding(addr) for addr in addresses])


In [27]:
embeddings[:1]

array([[-2.76040286e-01,  7.13616967e-01,  2.19237283e-01,
        -9.96102486e-03,  1.98695749e-01, -2.42680162e-01,
        -1.41236186e-03,  7.66457617e-01,  4.45752203e-01,
        -5.87909579e-01, -6.53534114e-01,  5.35354838e-02,
         5.68190180e-02,  5.28195262e-01,  6.54254913e-01,
        -1.05755043e+00,  1.72109529e-01, -1.93215296e-01,
         6.32912815e-01,  9.57220256e-01,  9.53001305e-02,
        -6.68093383e-01,  1.27110243e+00, -1.85492709e-01,
        -1.06712408e-01, -3.15679349e-02,  2.73533672e-01,
        -2.21952349e-01, -6.96013987e-01, -9.30691808e-02,
         1.32656932e-01, -1.67013362e-01, -5.44235945e-01,
         3.15085530e-01,  2.03791887e-01,  3.24675769e-01,
        -2.20295772e-01,  4.68534768e-01,  5.18212199e-01,
        -6.15812659e-01, -7.46970117e-01, -2.20482290e-01,
         2.40504351e-02, -9.52410161e-01, -8.22649419e-01,
         3.28472465e-01, -1.58717409e-01,  7.19089329e-01,
         3.81463200e-01,  2.99727410e-01,  7.22750127e-0

In [28]:
dim = embeddings.shape[1]
annoy_index = AnnoyIndex(dim, 'angular')
for i, embedding in enumerate(embeddings):
    annoy_index.add_item(i, embedding)

annoy_index.build(10)

True

In [29]:
def normalize_scores(scores):
    max_score = max(scores)
    return [score / max_score for score in scores]

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371000
    return R * c

def retrieve_neighbours(query, query_coords, top_n=5, bandwidth=200):
    query_embedding = get_embedding(query).flatten()
    indices, distances = annoy_index.get_nns_by_vector(query_embedding, top_n, include_distances=True)
    normalized_scores = normalize_scores([1 - dist for dist in distances])

    threshold = 0.25
    pruned_indices = [i for i, score in zip(indices, normalized_scores) if score >= threshold]
    retrieved_addresses = [addresses[i] for i in pruned_indices]
    retrieved_coords = [lat_lngs[i] for i in pruned_indices]
    pruned_scores = [score for score in normalized_scores if score >= threshold]


    haversine_distances = np.array([haversine(query_coords[0], query_coords[1], coord[0], coord[1]) for coord in retrieved_coords])


    kde = gaussian_kde(haversine_distances, bw_method=bandwidth / 1000)
    kde_scores = kde(haversine_distances)


    max_kde_score = max(kde_scores)
    normalized_kde_scores = [score / max_kde_score for score in kde_scores]

    return list(zip(retrieved_addresses, retrieved_coords, pruned_scores, normalized_kde_scores))





In [30]:
query_address = 'Honganuru channapattana urdu school land matk tipu carkal'
query_coords = [12.607466	, 77.216404]


neighbors = retrieve_neighbours(query_address, query_coords)
for neighbor in neighbors:
    print(f"Address: {neighbor[0]}, Coordinates: {neighbor[1]}, Score: {neighbor[2]}, KDE Score: {neighbor[3]}")

Address:  002 diamond residency kasavana halli carmelram post bangalore 560035 near st norbert church, Coordinates: [12.90237935, 77.61956552], Score: 1.0, KDE Score: 0.3328099878747833
Address:  006 vinayaka residency norbert church road kasavanahalli, Coordinates: [12.9113622, 77.678803], Score: 0.9959482001978, KDE Score: 0.9969157774512586
Address:  005 vinayaka residency kasavanahalli village behind bata showroom, Coordinates: [12.9113825, 77.67952829999999], Score: 0.9518729021699814, KDE Score: 0.9997884356152765
Address:  001 kaveri homes norbert church road kasavanahalli bangalore kasavanahalli, Coordinates: [12.911003, 77.67973620000002], Score: 0.9463493015032288, KDE Score: 1.0
Address:  00 devara krupa nilaya janatha colony doddakannalli, Coordinates: [12.9121999, 77.6984718], Score: 0.9081640525206842, KDE Score: 0.34223073004013155


In [ ]:
df2 = pd.read_csv('my_data_test.csv')

In [ ]:
df2.head()

Address   Latitude  Longitude
0                    Sogala, Channapatna Subdistrict  12.543418  77.175678
1                                        Channapatna  12.710511  77.268506
2           GLPS Kannidoddi, Channapatna Subdistrict  12.594512  77.229484
3  Honganuru channapattana urdu school land matk ...  12.607466  77.216404
4                         Near dairy, keladage beedi  12.562599  77.221361

In [ ]:
df2['Address'][3]

'Honganuru channapattana urdu school land matk tipu carkal'